In [95]:
import requests
import pandas as pd

In [96]:
url = "https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N"

# GET Website HTML

In [97]:
wiki = requests.get(url).text
soup = BeautifulSoup(wiki,'lxml')

# Scrap the Table

In [98]:
table = soup.find('table',{'class':'wikitable sortable'})

In [99]:
table_data = table.find_all('td')
pos_code = []
boro = []
neigh = []
for i in range(len(table_data)):
    data = table_data[i].text

    if data != "":
        if i % 3 == 0:
            pos_code.append(data)
        elif data.endswith("\n"):
            neigh.append(data.strip())
        else:
            boro.append(data)

In [100]:
df = pd.DataFrame()
df["PostCode"] = pos_code
df["Borough"] = boro
df["Neighbourhood"] = neigh
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Drop Not Assigned Boroughs

In [101]:
df = df[df.Borough != "Not assigned"]

# Other Preprocessing

In [102]:
pos_code = []
boro = []
neigh = []
for code in df.PostCode:
    pos_code.append(code)
    borough = df[df.PostCode == code].Borough.tolist()[0]
    neighborhood = ", ".join(df[df.PostCode == code].Neighbourhood.tolist())
    if "Not assigned" in neighborhood:
        neighborhood = borough
    boro.append(borough)
    neigh.append(neighborhood)

In [103]:
df_new = pd.DataFrame()
df_new["PostCode"] = pos_code
df_new["Borough"] = boro
df_new["Neighbourhood"] = neigh
df_new.drop_duplicates('PostCode', inplace=True)
df_new.reset_index(inplace=True)
df_new.drop(columns=['index'], inplace=True)

In [106]:
df_new.to_csv("toronto.csv")

In [105]:
df_new.shape

(103, 3)

# Geocoder Doesn't Work but the code is below

In [ ]:
import geocoder # import geocoder

lats = []
longs = []

# loop until you get the coordinates
for code in df_new.PostCode:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(code))
        lat_lng_coords = g.latlng
    lats.append(lat_lng_coords[0])
    longs.append(lat_lng_coords[1])

# Read Geo Coordinates

In [111]:
geo_codes = pd.read_csv("Geospatial_Coordinates.csv")

In [114]:
geo_codes.columns = ["PostCode", "Latitude", "Longitude"]

# Merge them with the Toronto Neighbourhood Data

In [125]:
df_new.PostCode = df_new.PostCode.astype(str)
geo_codes.PostCode = geo_codes.PostCode.astype(str)
df_final = df_new.merge(geo_codes, on="PostCode")

In [126]:
df_final.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [127]:
df_final.shape

(103, 5)